In [44]:
import numpy
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt

import glob
import os
import csv
from pprint import pprint
from pathlib import Path
import xml.etree.ElementTree as ET

from IPython.display import Image
from IPython.core.display import HTML

# Remove numpy warnings
import warnings
warnings.filterwarnings('ignore')

### mAP

Bounding boxes are a crude approximation of many objects. 
Ground truth boxes :  Box that denote the true position of an object (x,y,width,height).

### IoU (Intersection over Union)
It is the measure of the ratio between the intersection and the union of the predicted box and the ground truth boxes.[1]


In [45]:
Image(url= "https://tarangshah.com/blog/images/map-7.png", width=300, height=300)

### Precision & Recall

True Positive
False Positive
True Negative
False Negative

Total Number of Objects

Precision : 
Recall :

In [46]:
def read_annotation(annotation_path:str):
    status = True
    
    tree = ET.parse(annotation_path)
    root = tree.getroot()

    all_results = []
    for boxes in root.iter('object'):
        single_result = {}

        filename = root.find('filename').text
        class_name = boxes.find('name').text
        
        ymin, xmin, ymax, xmax = None, None, None, None

        for box in boxes.findall("bndbox"):
            ymin = int(box.find("ymin").text)
            xmin = int(box.find("xmin").text)
            ymax = int(box.find("ymax").text)
            xmax = int(box.find("xmax").text)

        list_with_single_boxes = [xmin, ymin, xmax, ymax]
        single_result['label'] = class_name
        single_result['bbox'] = list_with_single_boxes
        
        all_results.append(single_result)

    result = {}
    result["status"] = status 
    result["results"] = all_results 
    
    return result

result = read_annotation("/home/jayasimha/Pictures/door_2.xml")
pprint(result)

{'results': [{'bbox': [80, 40, 144, 185], 'label': 'door'},
             {'bbox': [34, 6, 67, 55], 'label': 'lamp'}],
 'status': True}


In [51]:
class ObjDetector():
    def __init__(self):
        self.graph_def = None
        self.out = None
        self.img = None
        self.labels = {}
    
    def initialize_model(self,pb_path:str):
        with tf.gfile.FastGFile(pb_path, 'rb') as f:
            self.graph_def = tf.GraphDef()
            self.graph_def.ParseFromString(f.read())
            
    def set_labels(self,path:str):
        with open(path, 'r') as csvFile:
            reader = csv.reader(csvFile)
            for i,row in enumerate(reader):
                print(i,row)
                self.labels[i+1] = row[-1]
        csvFile.close()
    
    def run_detection(self,img_path:str):
        with tf.Session() as sess:
            # Restore session
            sess.graph.as_default()
            tf.import_graph_def(self.graph_def, name='')

            # Read and preprocess an image.
            self.img = cv2.imread(img_path)
            self.img = cv2.resize(self.img,(480,640))
    
            rows = self.img.shape[0]
            cols = self.img.shape[1]
            inp = cv2.resize(self.img, (300, 300))
            inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

            # Run the model
            self.out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                            sess.graph.get_tensor_by_name('detection_scores:0'),
                            sess.graph.get_tensor_by_name('detection_boxes:0'),
                            sess.graph.get_tensor_by_name('detection_classes:0')],
                           feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
        
        return self
                
    
    def get_result(self):
        status = False
        
        all_results = []
        # Visualize detected bounding boxes.
        num_detections = int(self.out[0][0])
        for i in range(num_detections):
            single_result = {}

            classId = int(self.out[3][0][i])
            score = float(self.out[1][0][i])
            bbox = [float(v) for v in out[2][0][i]]
            if score > 0.3:
                x = bbox[1] * cols
                y = bbox[0] * rows
                right = bbox[3] * cols
                bottom = bbox[2] * rows
                
                cv2.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), 
                              (125, 255, 51), thickness=2)
                
                single_result['label'] = self.labels[classId]
                single_result['bbox'] = [x,y,x+right,y+bottom]
                
                all_results.append[single_result]
                
        result = {}
        result["status"] = status 
        result["results"] = all_results
    
        return img , result

In [52]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [53]:
TF_MODEL_PATH = "/home/jayasimha/NJ/Models/faster_rcnn_inception_resnet_v2_atrous_oid_v4_2018_12_12/frozen_inference_graph.pb"
MODEL_LABEL_PATH = "/home/jayasimha/GitHub/Research-Development-HBRS/src/utils/class-descriptions-boxable.csv"

DATASET_PATH = "/home/jayasimha/Pictures/" 

obj_detector = ObjDetector()
obj_detector.initialize_model(TF_MODEL_PATH)
obj_detector.set_labels(MODEL_LABEL_PATH)

0 ['/m/011k07', 'Tortoise']
1 ['/m/011q46kg', 'Container']
2 ['/m/012074', 'Magpie']
3 ['/m/0120dh', 'Sea turtle']
4 ['/m/01226z', 'Football']
5 ['/m/012n7d', 'Ambulance']
6 ['/m/012w5l', 'Ladder']
7 ['/m/012xff', 'Toothbrush']
8 ['/m/012ysf', 'Syringe']
9 ['/m/0130jx', 'Sink']
10 ['/m/0138tl', 'Toy']
11 ['/m/013y1f', 'Organ']
12 ['/m/01432t', 'Cassette deck']
13 ['/m/014j1m', 'Apple']
14 ['/m/014sv8', 'Human eye']
15 ['/m/014trl', 'Cosmetics']
16 ['/m/014y4n', 'Paddle']
17 ['/m/0152hh', 'Snowman']
18 ['/m/01599', 'Beer']
19 ['/m/01_5g', 'Chopsticks']
20 ['/m/015h_t', 'Human beard']
21 ['/m/015p6', 'Bird']
22 ['/m/015qbp', 'Parking meter']
23 ['/m/015qff', 'Traffic light']
24 ['/m/015wgc', 'Croissant']
25 ['/m/015x4r', 'Cucumber']
26 ['/m/015x5n', 'Radish']
27 ['/m/0162_1', 'Towel']
28 ['/m/0167gd', 'Doll']
29 ['/m/016m2d', 'Skull']
30 ['/m/0174k2', 'Washing machine']
31 ['/m/0174n1', 'Glove']
32 ['/m/0175cv', 'Tick']
33 ['/m/0176mf', 'Belt']
34 ['/m/017ftj', 'Sunglasses']
35 ['/m/018j

In [54]:
img_file_list = glob.glob( DATASET_PATH + "/*.jpg", recursive=True)
    
for img in img_file_list :
    name = Path(img).stem
    img_path = DATASET_PATH + name + ".jpg"
    annotation_path = DATASET_PATH + name + ".xml"

    print(img_path,annotation_path)
    
        
    _ , detection_result = obj_detector.run_detection(img).get_result()
    annotation_result = read_annotation(annotation_path)
    
    print(detection_result)
    break


/home/jayasimha/Pictures/door.jpg /home/jayasimha/Pictures/door.xml


TypeError: 'NoneType' object is not subscriptable

### References

1. https://tarangshah.com/blog/2018-01-27/what-is-map-understanding-the-statistic-of-choice-for-comparing-object-detection-models/
2. https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
3. http://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf
4. http://cocodataset.org/#detection-eval
5. https://stackoverflow.com/questions/53317592/reading-pascal-voc-annotations-in-python